In [1]:
import ufl
from ufl import  TrialFunction, TestFunction, FacetNormal, lhs, rhs,SpatialCoordinate

import dolfinx
from dolfinx import mesh, fem, io
from dolfinx.fem import FunctionSpace, VectorFunctionSpace, locate_dofs_geometrical, form
from dolfinx.fem.petsc import LinearProblem

from mpi4py import MPI
import petsc4py.PETSc
import numpy as np
import matplotlib.pyplot as plt


# Operators:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


dot = Infix(ufl.dot)
inner = Infix(ufl.inner)

def vector(*args):
    return ufl.as_vector(tuple(args))

from ufl import exp,sym,tr,sqrt
from ufl import nabla_div as div
from ufl import nabla_grad as grad
from ufl import as_matrix as matrix

npor = Infix(np.logical_or)
npand = Infix(np.logical_and)

def I(func_like):
    """Create matrix Identity dimension of func_like

    Args:
        func_like (Function): Give geometric dimension

    Returns:
        Tensor: Identity
    """
    return ufl.Identity(func_like.geometric_dimension())

# Classes:

def Function(space, func=None):
    """Function on new space. Without func just create a Function on space. 
    Args:
        space (FunctionSpace): New space
        function (): 
    \nScalars - fem.Function,fem.Constant, ufl_function, callable function, number
    \nVectors - fem.vector_Function,fem.vector_Constant, ufl_vector_function, callable vector_function, tuple_number
    Returns:
        fem.Function: Function
    """

    result = fem.Function(space)
    tupe = str(func.__class__)[8:-2]
    cord = SpatialCoordinate(space)
    cargo_vector = vector(*map(lambda x, y: x - y, cord, cord))

    # None
    if func == None:
        return result

    # fem.Function
    elif tupe == ('dolfinx.fem.function.Function'):
        expression = func

    # fem.Constant
    elif tupe == ('dolfinx.fem.function.Constant'):
        if len(func.ufl_shape) == 0 : func2 = func.value + (cord[0] - cord[0])
        else: func2 = vector(*func.value) + cargo_vector
        expression = fem.Expression(func2, space.element.interpolation_points())

    # ufl object
    elif tupe[:3] == 'ufl':
        if len(func.ufl_shape) !=0: func2 = func + cargo_vector
        else: func2 = func
        expression = fem.Expression(func2, space.element.interpolation_points())

    # Python function
    elif tupe == ('function'): expression = func
 
    # Number
    elif not hasattr(func, '__call__'):
        if hasattr(func,'__getitem__'): func2 =  vector(*func) + cargo_vector
        else: func2 = func + (cord[0] - cord[0])
        expression = fem.Expression(func2, space.element.interpolation_points())


    result.interpolate(expression)
    return result


def Constant(space, const):
    """Constant on space

    Args:
        space (fem.FunctionSpace): Space
        const (auny number): Any number

    Returns:
        fem.function.Constant: Constant on space
    """
    return fem.Constant(space, petsc4py.PETSc.ScalarType(const))


def create_facet_markers(domain, bound_markers):
    """Mark facets under conditious

    Args:
        domain (Domain): Domain
        bound_markers (mark,python_function): List of mark and function
    
    Return:
        tags(mesh.meshtags): Marked facets
    """
    facet_indices, facet_markers = [], []
    for (marker, condition) in bound_markers:
        facets = mesh.locate_entities(
            domain,
            domain.topology.dim - 1,
            condition,
            )
        facet_indices.append(facets)
        facet_markers.append(np.full_like(facets, marker))
    facet_indices = np.hstack(facet_indices).astype(np.int32)
    facet_markers = np.hstack(facet_markers).astype(np.int32)
    sorted_facets = np.argsort(facet_indices)
    facet_tags = mesh.meshtags(
        domain,
        domain.topology.dim - 1,
        facet_indices[sorted_facets],
        facet_markers[sorted_facets],
        )

    return facet_tags


def create_connectivity(domain):
    """Need to compute facets to Boundary value

    Args:
        domain (Mesh): Domain
    """
    domain.topology.create_connectivity(
        domain.topology.dim - 1,
        domain.topology.dim,
        )


def DirichletBC(space, func, combined_marker):
    """Create Dirichlet condition

    Args:
        space (fem.FunctionSpace): Function space
        func (fem.function): fem.Function
        combined_marker (Any): One from next
        \nFunction - boundary marker function
        \nAll - all boundary
        \n(mesh.meshtags, marker) - list or tuple, marker of boundary from Marked_facets - mesh.meshtags
        
    Returns:
        condition (dirichletbc): Dirichlet condition
    """

    def get_exterior_facets():
        exterior_facets = mesh.exterior_facet_indices(domain.topology)
        return exterior_facets

    def all_dirichlet(dofs, func):
        if hasattr(func, 'function_space'):
            bc = fem.dirichletbc(dofs=dofs, value=func)
        else:
            bc = fem.dirichletbc(V=space, dofs=dofs, value=func)
        return bc

    domain = space.mesh

    if combined_marker == 'All':
        facets = get_exterior_facets()
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )

    elif type(combined_marker) == (tuple or list):
        marked_facets, marker = combined_marker
        facets = marked_facets.find(marker)
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )
    else:
        dofs = fem.locate_dofs_geometrical(space, marker=combined_marker)

    bc = all_dirichlet(dofs, func)

    return bc


# Post processing:
def errors_L(uS, uEx):
    """Compute error norm on boundary

    Args:
        uS (Function): Numeric solution
        uEx (Function): Exact or model solution

    Returns:
        List: L1 and L2 norms
    """
    domain = uS.function_space.mesh

    V_err = FunctionSpace(
        domain,
        ('CG', uS.function_space.element.space_dimension),
        )
    uEx = Function(V_err, uEx)
    L1_scalar = fem.assemble_scalar(form((uS-uEx) * dx))
    L2_scalar = fem.assemble_scalar(form((uS - uEx)**2 * dx))

    L1_err = np.abs(domain.comm.allreduce(L1_scalar, op=MPI.SUM))
    L2_err = np.sqrt(domain.comm.allreduce(L2_scalar, op=MPI.SUM))
    return (L1_err, L2_err)


def line_collision(domain, line_cord):
    """Generate points and cells of colliding domain and line

    Args:
        domain (mesh): Domain
        line_cord (array): 3D line contervertor of coordinates 

    Returns:
        Tuple: Collision points and collision cells
    """
    bb_tree = dolfinx.geometry.BoundingBoxTree(domain, domain.topology.dim)

    cells_on_line = []
    points_on_line = []
    cell_candidates = dolfinx.geometry.compute_collisions(bb_tree, line_cord.T)
    colliding_cells = dolfinx.geometry.compute_colliding_cells(
        domain, cell_candidates, line_cord.T
        )
    for i, point in enumerate(line_cord.T):
        if len(colliding_cells.links(i)) > 0:
            points_on_line.append(point)
            cells_on_line.append(colliding_cells.links(i)[0])

    points_on_line = np.array(points_on_line, dtype=np.float64)

    return (points_on_line, cells_on_line)


def graph2D(fig, lists, natural_show=False, points_on=False):
    """Create graph from fem.Function

    Args:
        fig (plt.Figure): Figure
        lists (fem.Function , plt.Axes, str): List of (u, curent axes, title)
        method (bool): Graph method True = tripcolor, False = tricontourf
    """

    def data_construct(dofs, x_array):
        data = np.column_stack((dofs[:, 0:2], x_array))
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        return [x_data, y_data, z_data]

    for list in lists:
        fig, ax = plt.subplots()
        plt.close()
        u, ax, title = list
        dofs = u.function_space.tabulate_dof_coordinates()
        ax.set_title(title)
        data = data_construct(dofs, u.x.array)

        if points_on: ax.plot(data[0], data[1], 'o', markersize=2, color='grey')

        if natural_show:
            plot = ax.tripcolor(*data)
        else:
            try:
                levels = np.linspace(u.x.array.min(), u.x.array.max(), 10)
                plot = ax.tricontourf(
                    *data,
                    levels=levels,
                    )
            except:
                print(f'{title} - error')

        ax.set_aspect(1)
        fig.colorbar(plot, ax=ax)
    return

In [11]:
L = 1
W = 0.2
domain = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, W, W])],
    [10, 3, 3],
    cell_type=mesh.CellType.hexahedron,
    )
create_connectivity(domain=domain)
V = VectorFunctionSpace(domain, ('CG', 1))

u, v = TrialFunction(V), TestFunction(V)
x, y, z = SpatialCoordinate(V)
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure('cell', subdomain_id='everywhere')

delta = 0.4 * (W / L)**2
lamb = 1.25
mu = 1
rho = 2

T = vector(0, 0.01, -0.02)
uD1 = vector(0, 0, 0)
uD2 = vector(0, 0, -0.2)
f = vector(0, 0, -rho * delta)
epsilon = lambda u: sym(grad(u))
sigma = lambda u: lamb * div(u) * I(u) + 2 * mu * epsilon(u)


bcs = [
    DirichletBC(V, Function(V,uD1), lambda x: np.isclose(x[0], 0)),
    DirichletBC(V, Function(V,uD2), lambda x: np.isclose(x[0], 1)),
    ]


F = (sigma(u)|inner|epsilon(v)) * dx - (f|dot|v) * dx - (T|dot|v) * ds

In [12]:
problem = LinearProblem(
    lhs(F),
    rhs(F),
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly", "pc_type": "lu"
        },
    )
uS = problem.solve()

In [13]:
s = sigma(uS) - 1 / 3 * tr(sigma(uS)) * I(uS)
estress = sqrt(3 / 2 * s**2)

V_von = FunctionSpace(domain, ('CG', 1))
stress = Function(V_von, estress)

In [14]:
import shutil
try:
    shutil.rmtree('/home/VTK/Elasticity_files')
except:
    print('Directory empty yet')

In [15]:
with io.VTKFile (
    domain.comm,
    '/home/VTK/Elasticity_files/elasticity.pvd',
    'w'
) as file:
    # file.write_mesh(domain)
    uS.name = 'Deformation'
    stress.name = 'Stress von Mises'
    # Tf.name = 'External power'
    file.write_function([uS,stress])

# Problem with div grad

In [16]:
testf2 = Function(V, div(sym(grad(uS))))
testf2.x.array

array([ 8.01590524, -0.25891896,  7.06313568, ...,  2.90067109,
       -1.74471223,  4.77612218])

In [17]:
testf3 = Function(V, div(grad(uS)))
testf3.x.array

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
type(sym(grad(uS)))


ufl.tensoralgebra.Sym

In [19]:
type(grad(uS))

ufl.differentiation.NablaGrad